# 01 — Estimands and Assumptions

CausalRL requires you to be explicit about **what** you estimate (the estimand)
and **which assumptions** identify that estimand. This notebook demonstrates:

- How to define a `PolicyValueEstimand`.
- How assumptions are enforced by estimators.
- How diagnostics flag overlap violations.

## Setup

Suggested environment:

```
pip install ".[plots]"
```

In [1]:
from __future__ import annotations

import numpy as np

from crl.assumptions import Assumption, AssumptionSet
from crl.assumptions_catalog import (
    CORRECT_MODEL,
    MARKOV,
    OVERLAP,
    SEQUENTIAL_IGNORABILITY,
)
from crl.benchmarks.bandit_synth import SyntheticBandit, SyntheticBanditConfig
from crl.diagnostics.plots import plot_weight_histogram
from crl.estimands.policy_value import PolicyContrastEstimand, PolicyValueEstimand
from crl.estimators.importance_sampling import ISEstimator
from crl.policies import UniformPolicy
from crl.utils.seeding import set_seed
from crl.viz import configure_notebook_display

In [2]:
set_seed(0)
np.random.seed(0)
configure_notebook_display()

## Custom assumptions

Assumptions are explicit, typed objects. You can add your own and mix them
with catalog defaults.

In [3]:
benchmark = SyntheticBandit(SyntheticBanditConfig(seed=0))
dataset = benchmark.sample(num_samples=500, seed=1)

stationarity = Assumption(
    name="stationarity",
    description="The environment dynamics do not drift during collection.",
)

assumptions = AssumptionSet([SEQUENTIAL_IGNORABILITY, OVERLAP, stationarity])
assumptions.to_dict()

{'sequential_ignorability': {'name': 'sequential_ignorability',
  'description': 'No unmeasured confounding given observed history.'},
 'overlap': {'name': 'overlap',
  'description': 'Behavior policy has support over target policy actions.'},
 'stationarity': {'name': 'stationarity',
  'description': 'The environment dynamics do not drift during collection.'}}

The catalog entries are reusable primitives for building checklists.

In [4]:
AssumptionSet([MARKOV]).names()

['markov']

## Define an estimand

The policy value estimand encodes the target policy, horizon, discount, and
identification assumptions. Estimators check these assumptions before
running.

In [5]:
estimand = PolicyValueEstimand(
    policy=benchmark.target_policy,
    discount=1.0,
    horizon=1,
    assumptions=assumptions,
)

estimand

PolicyValueEstimand(policy=TabularPolicy, discount=1.0, horizon=1, assumptions=['overlap', 'sequential_ignorability', 'stationarity'])

## Policy contrast estimand

A contrast estimand expresses the difference between two policy values.

In [6]:
control_policy = UniformPolicy(action_space_n=dataset.action_space_n)
control_estimand = PolicyValueEstimand(
    policy=control_policy,
    discount=1.0,
    horizon=1,
    assumptions=AssumptionSet([SEQUENTIAL_IGNORABILITY, OVERLAP]),
)

contrast = PolicyContrastEstimand(treatment=estimand, control=control_estimand)
contrast.to_dict()

{'treatment': {'policy': 'TabularPolicy',
  'discount': 1.0,
  'horizon': 1,
  'assumptions': ['overlap', 'sequential_ignorability', 'stationarity']},
 'control': {'policy': 'UniformPolicy',
  'discount': 1.0,
  'horizon': 1,
  'assumptions': ['overlap', 'sequential_ignorability']}}

Model-based estimators typically add a correct-model assumption.

In [7]:
AssumptionSet([CORRECT_MODEL]).names()

['correct_model']

## Overlap violations

If the logging policy never takes some actions that the target policy would
take, importance-weighted estimators become unstable. We'll create a toy
overlap violation and inspect the diagnostics.

In [8]:
dataset_bad = benchmark.sample(num_samples=500, seed=2)
dataset_bad.behavior_action_probs = np.clip(
    dataset_bad.behavior_action_probs, 0.0, 0.02
)

report = ISEstimator(estimand).estimate(dataset_bad)
report.diagnostics

{'overlap': {'min_behavior_prob': 0.02,
  'min_target_prob': 0.038790349205150024,
  'fraction_behavior_below_threshold': 0.0,
  'fraction_target_below_threshold': 0.0,
  'ratio_min': 1.9395174602575012,
  'ratio_max': 35.57378517851286,
  'ratio_q50': 11.083778973023602,
  'ratio_q90': 28.88163828666249,
  'ratio_q99': 35.57378517851286,
  'support_violations': 0},
 'ess': {'ess': 291.0519313570731, 'ess_ratio': 0.5821038627141462},
 'weights': {'min': 1.9395174602575012,
  'max': 35.57378517851286,
  'mean': 13.368479302599834,
  'std': 11.327027264041313,
  'q95': 35.57378517851286,
  'q99': 35.57378517851286,
  'skew': 0.6636527695733089,
  'kurtosis': -0.9647032937905231,
  'tail_fraction': 0.508},
 'max_weight': 35.57378517851286,
 'model': {}}

## Results snapshot

The overlap stress test produces large weights. We summarize the estimate and
visualize the weight distribution.

In [ ]:
summary = report.to_dataframe()
print(
    summary[["value", "lower_bound", "upper_bound"]]
    .round(3)
    .to_string(index=False)
)
summary

In [ ]:
weights = (
    benchmark.target_policy.action_prob(dataset_bad.contexts, dataset_bad.actions)
    / dataset_bad.behavior_action_probs
)
fig_weights = plot_weight_histogram(
    weights,
    log_y=True,
    title="Weight histogram under overlap stress",
    clip_quantile=0.995,
)
fig_weights

## Takeaways

- Estimands make assumptions explicit and enforceable.
- Diagnostics are tied to assumptions (e.g., overlap).
- When overlap is poor, estimators warn and flag the assumption.